In [1]:
import pandas
import numpy

In [2]:
motor = 1400 #rpm

table = pandas.read_table("table.txt")
table['ratio'] = table.spindle / motor
table

,spindle,d1,d2,d3,d4,ratio
0,110,45,a,d,184,0.078571
1,190,45,a,d,159,0.135714
2,220,65,b,d,184,0.157143
3,320,45,a,b,136,0.228571
4,380,65,b,c,159,0.271429
5,430,90,c,d,184,0.307143
6,860,65,b,a,117,0.614286
7,1170,90,c,b,136,0.835714
8,1260,99,d,c,159,0.900000
9,1680,90,c,a,117,1.200000


Kiekvieną lentelės eilutę atitinka viena tiesinė lygtis.

$R = \frac{d_1}{d_2} \frac{d_3}{d_4}$

$d_2 = \frac{d_1 d_3}{R d_4}$

$d_2 - \frac{d_1}{R d_4} d_3 = 0$

Fiksuojame A ir sudarome lygčių sistemą su trim nežinomaisiais: B, C, D. Lygčių sistemos matrica:

In [24]:
indices = dict(zip("bcda", range(4)))

d_a = 154

def buildrow(spindle, d1, d2, d3, d4, ratio, *args):
    i2 = indices[d2]
    i3 = indices[d3]
    row = numpy.zeros(len(indices))
    row[i2] = 1
    row[i3] = -d1 / (ratio * d4)
    if d2 == 'a':
        row[i2] *= -d_a
    if d3 == 'a':
        row[i3] *= -d_a
    return row

coeffs = numpy.matrix([buildrow(*r) for i, r in table.iterrows()])
coeffs

matrix([[   0.        ,    0.        ,   -3.11264822, -154.        ],
        [   0.        ,    0.        ,   -2.08540218, -154.        ],
        [   1.        ,    0.        ,   -2.24802372,    0.        ],
        [  -1.44761029,    0.        ,    0.        , -154.        ],
        [   1.        ,   -1.5061238 ,    0.        ,    0.        ],
        [   0.        ,    1.        ,   -1.59251769,    0.        ],
        [   1.        ,    0.        ,    0.        ,  139.27648579],
        [  -0.7918552 ,    1.        ,    0.        ,    0.        ],
        [   0.        ,   -0.6918239 ,    1.        ,    0.        ],
        [   0.        ,    1.        ,    0.        ,   98.71794872],
        [  -0.50955882,    0.        ,    1.        ,    0.        ],
        [   0.        ,    0.        ,    1.        ,   63.34401709]])

In [25]:
n = len(indices)
solution, residuals, rank, singular = numpy.linalg.lstsq(coeffs[:,0:n - 1], coeffs[:,n - 1])
print(solution.T, residuals, rank, singular)

ds = pandas.Series([d_a] + list(solution.flat), index=list("abcd"))
ds

[[ 120.44942012   87.88298672   56.71564395]] [[ 3006.75267969]] 3 [ 5.02210209  2.85590723  1.72591385]


a    154.000000
b    120.449420
c     87.882987
d     56.715644
dtype: float64

Gauti dydžiai suapvalinami iki milimetrų:

In [26]:
ds = numpy.round(ds)
ds

a    154
b    120
c     88
d     57
dtype: float64

Gauti greičiai stulpelyje actual_spindle:

In [23]:
dsdict = dict(ds)
table['actual_ratio'] = [r.d1 / dsdict[r.d2] * dsdict[r.d3] / r.d4 for i, r in table.iterrows()]
table['actual_spindle'] = table['actual_ratio'] * motor
table

,spindle,d1,d2,d3,d4,ratio,actual_ratio,actual_spindle
0,110,45,a,d,184,0.078571,0.090383,126.535917
1,190,45,a,d,159,0.135714,0.104594,146.431501
2,220,65,b,d,184,0.157143,0.166818,233.544686
3,320,45,a,b,136,0.228571,0.258951,362.531969
4,380,65,b,c,159,0.271429,0.298087,417.321803
5,430,90,c,d,184,0.307143,0.316770,443.478261
6,860,65,b,a,117,0.614286,0.709877,993.827160
7,1170,90,c,b,136,0.835714,0.907563,1270.588235
8,1260,99,d,c,159,0.900000,0.961432,1346.004440
9,1680,90,c,a,117,1.200000,1.347985,1887.179487
